In [1]:
from transformers import BertTokenizer,TFBertModel,BertConfig
import tensorflow as tf
import time
import numpy as np

In [2]:
import pickle

#Training
with open('../Patent_data/labels.pickle', 'rb') as file:
    labels =pickle.load(file)
with open('../Patent_data/tokens.pickle', 'rb') as file:
    tokens =pickle.load(file)



In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
max_length_test = 20
test_sentence = tokens[0]

tokenized = tokenizer.tokenize(test_sentence)
print('tokenized', tokenized)


tokenized ['用', '於', '半', '導', '體', '裝', '置', '之', '黏', '合', '劑', '膜', '本', '發', '明', '提', '供', '了', '一', '種', '用', '於', '半', '導', '體', '裝', '置', '的', '黏', '合', '劑', '膜', '，', '所', '述', '黏', '合', '劑', '膜', '包', '括', '在', '0', '℃', '至', '5', '℃', '下', '具', '有', '50', 'μ', 'm', '/', 'm', '‧', '℃', '至', '150', 'μ', 'm', '/', 'm', '‧', '℃', '的', '線', '性', '膨', '脹', '係', '數', '的', '基', '膜', '。', '所', '述', '黏', '合', '劑', '膜', '在', '低', '溫', '儲', '存', '較', '長', '時', '間', '後', '具', '有', '優', '異', '的', '捲', '繞', '形', '狀', '穩', '定', '性', '，', '從', '而', '在', '以', '後', '的', '半', '導', '體', '封', '裝', '工', '藝', '中', '不', '會', '導', '致', '傾', '斜', '現', '象', '並', '降', '低', '缺', '陷', '。']


In [10]:
bert_input = tokenizer.encode_plus(
                        test_sentence,                      
                        add_special_tokens = True, # add [CLS], [SEP]
                        max_length = 150, # max length of the text that can go to BERT
                        pad_to_max_length = True, # add [PAD] tokens
                        return_attention_mask = True, # add attention mask to not focus on pad tokens
              )
print('encoded', bert_input)

encoded {'input_ids': [101, 4500, 3176, 1288, 2206, 7768, 6172, 5390, 722, 7945, 1394, 1212, 5606, 3315, 4634, 3209, 2990, 897, 749, 671, 4934, 4500, 3176, 1288, 2206, 7768, 6172, 5390, 4638, 7945, 1394, 1212, 5606, 8024, 2792, 6835, 7945, 1394, 1212, 5606, 1259, 2886, 1762, 121, 360, 5635, 126, 360, 678, 1072, 3300, 8145, 220, 155, 120, 155, 342, 360, 5635, 8269, 220, 155, 120, 155, 342, 360, 4638, 5221, 2595, 5610, 5568, 913, 3149, 4638, 1825, 5606, 511, 2792, 6835, 7945, 1394, 1212, 5606, 1762, 856, 3984, 1033, 2100, 6733, 7269, 3229, 7279, 2527, 1072, 3300, 1032, 4530, 4638, 2947, 5254, 2501, 4311, 4952, 2137, 2595, 8024, 2537, 5445, 1762, 809, 2527, 4638, 1288, 2206, 7768, 2196, 6172, 2339, 5971, 704, 679, 3298, 2206, 5636, 1005, 3162, 4412, 6496, 699, 7360, 856, 5375, 7379, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
vocab = tokenizer.vocab
for index in bert_input['input_ids']:
    print(list(vocab)[index],end=',')

[CLS],用,於,半,導,體,裝,置,之,黏,合,劑,膜,本,發,明,提,供,了,一,種,用,於,半,導,體,裝,置,的,黏,合,劑,膜,，,所,述,黏,合,劑,膜,包,括,在,0,℃,至,5,℃,下,具,有,50,μ,m,/,m,‧,℃,至,150,μ,m,/,m,‧,℃,的,線,性,膨,脹,係,數,的,基,膜,。,所,述,黏,合,劑,膜,在,低,溫,儲,存,較,長,時,間,後,具,有,優,異,的,捲,繞,形,狀,穩,定,性,，,從,而,在,以,後,的,半,導,體,封,裝,工,藝,中,不,會,導,致,傾,斜,現,象,並,降,低,缺,陷,。,[SEP],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],[PAD],

上述範例是設定字數=150字:
* 若文件不足150字就會以[SEP]結尾並padding至150
* 若文件超過150字，多餘的字數將不輸入，句尾(第151字)放置[SEP]

In [12]:
model = TFBertModel.from_pretrained("bert-base-chinese", output_hidden_states = True)

拿hidden states:  
+ outputs[2]是return hidden_states
+ bert-base-chinese hiddenstates維度13 * batch * max_length * 768  
+ 此範例是13 * 1 * 150 * 768

In [14]:
bert_input = tokenizer.encode_plus(
                    tokens[0],                      
                    add_special_tokens = True, # add [CLS], [SEP]
                    max_length = 150, # max length of the text that can go to BERT
                    pad_to_max_length = True, # add [PAD] tokens
                    return_attention_mask = True, # add attention mask to not focus on pad tokens
          )
input_ids = tf.constant(bert_input['input_ids'])[None, :]
outputs =  model(input_ids)
hidden_states = outputs[2]  #hidden_states dimension:[# layers, # batches, # tokens, # features]
hidden_states = tf.reshape(hidden_states,[13,-1,768]) #dimension:[# layers, # tokens, # features]
avg_doc_vector = tf.math.reduce_mean(hidden_states[-2],axis=0)
avg_doc_vector.shape

TensorShape([768])

avg_doc_vector拿到hidden_states倒數第二層Encoder的向量